<a href="https://colab.research.google.com/github/MustafaShama98/Cloud-Compute/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install firebase
!pip install nltk

In [3]:
from nltk.stem import PorterStemmer
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
from nltk.stem import PorterStemmer
import random
import firebase_admin
from firebase_admin import db, credentials
from IPython.display import display, HTML ,clear_output
import ast
import ipywidgets as widgets
from firebase import firebase

#Step1 : getting page document data
def fetch_page(url):
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
  }
  response = requests.get(url,headers=headers)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup
  else:
   return None

#Step 2 : Indexing info from document data
#example: {'game':572}
def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

#Step 3 : Text Processing
#a) removing connectors:

def remove_stop_words(index):
    stop_words = {'a', 'an', 'the', 'and', 'or',                    'in', 'on', 'at'}
    for stop_word in stop_words:
          if stop_word in index:
            del index[stop_word]
    return index

#b)putting the same words (eat, eating , ate....) in the same index
def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

#Step 4: the searching function
#examole : query= "game of thrones" , return value {'game': 583, 'of': 1188}
def search(query, index):
    stemmer = PorterStemmer()
    query_words = re.findall(r'\w+', query.lower())
    results = {}
    for word in query_words:
        word = stemmer.stem(word)
        if word in index:
            results[word] = index[word]
    return results


#The whole search function:
def search_engine(url, query):
    soup = fetch_page(url)
    if soup is None:
        return None
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    results = search(query, index)
    return results
#this function recives url string , and returns list of all links in it
def fetch_page_and_extract_links(url):
    response = requests.get(url)
    if response.status_code == 200: #checking if request has succeeded
        soup = BeautifulSoup(response.text, 'html.parser')
        links = [link.get('href') for link in soup.find_all('a', href=True)]
        return links
    else:
        return None

#this function recieves url string , and returns
def getListOfLinks():
  # Base URL for VMware
  base_url = 'https://www.vmware.com/'
  page_content = fetch_page('https://www.vmware.com/products.html')
  # Extracting h5 elements
  h5_elements = page_content.select('h5')

    # Create an empty dictionary to store the results
  result_dict = {}

  # Iterate through each h5 element
  for h5 in h5_elements:
      # Extract the 'a' tag from each h5 element
      a_tag = h5.find('a')

      # Check if 'a' tag is found
      if a_tag:
          # Extract text and href attributes
          text = a_tag.get_text(strip=True)
          href = a_tag['href']

          # Check if the link starts with 'http'
          if href.startswith('http'):
              link = href
          else:
              # If it starts with '/', append the base URL
              link = urljoin(base_url, href)

          # Add the information to the result dictionary
          result_dict[text] = {'Link': link, 'Occurrences': 0}

  # Convert the dictionary to a JSON-formatted string
  json_string = json.dumps(result_dict, indent=2)

  #Print the JSON object
  #print('8787')
  return result_dict

#this function checks if string is urllink
def is_url(string):
    # Regular expression pattern to match URLs
    url_pattern = re.compile(
        r'^(?:http|ftp)s?://'  # http:// or https:// or ftp://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|'  # domain...
        r'localhost|'  # localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'  # ...or ip
        r'(?::\d+)?'  # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

    return bool(re.match(url_pattern, string))

def remove_duplicates(d):
    unique_dict = []
    seen = set()
    for item in d:
        if (item['Link'], item['Rank']) not in seen:
            unique_dict.append(item)
            seen.add((item['Link'], item['Rank']))
    return unique_dict


indexes_table_name = '/vmware_indexes'
searched_words_table = '/searched_words'
#making the serach query func recives query string , returns list of dictioneries
def vmwareSearch(query):
  search_results = []
  res = FBconn.get(searched_words_table, None)
  for key in res:
     index= res[key] #res[key]= { .... , ....}
     searchResult =search(query, index)
     #ranking the result :
     rank=1
     for word, count in searchResult.items():
          rank = rank*1/count
     rank = 1-rank
     # Append to search_results list
     if rank!=0:
          search_results.append({
                  'Link': index['TheLinkIsHere'],
                  'Rank': rank,
                  'searchResult': searchResult
              })
  # Sort search_results by the 'Rank' key in descending order
  search_results = sorted(search_results, key=lambda x: x['Rank'], reverse=True)
  search_results=remove_duplicates(search_results)
  return search_results


#for each link in our website we will open anew folder in our data base that contains tle indexes info
FBconn = firebase.FirebaseApplication('https://cloudcomputing-2bbc8-default-rtdb.europe-west1.firebasedatabase.app/',None)
def getAllPages():
      ######
    #Saving each index in data base
    getIndexing= getListOfLinks()
    count=0
    max_iterations=35
    for key, value in getIndexing.items():
        linkstring =value['Link']
        if count >= max_iterations:
            break
        if(is_url(value['Link'])):
            soup = fetch_page(value['Link'])
            if soup is None:
                pass
            else :
              index = index_words(soup)
              index = apply_stemming(index)
              index = remove_stop_words(index)
              index['TheLinkIsHere'] = linkstring
              FBconn.post(searched_words_table, index)
              count += 1


import json
from urllib.parse import urljoin
def get_Important_words():
  # Base URL for VMware
  base_url = 'https://www.vmware.com/'
  page_content = fetch_page('https://www.vmware.com/products.html')
  # Extracting h5 elements
  h5_elements = page_content.select('h5')

    # Create an empty dictionary to store the results
  result_dict = {}

  # Iterate through each h5 element
  for h5 in h5_elements:
      # Extract the 'a' tag from each h5 element
      a_tag = h5.find('a')

      # Check if 'a' tag is found
      if a_tag:
          # Extract text and href attributes
          text = a_tag.get_text(strip=True)
          href = a_tag['href']

          # Check if the link starts with 'http'
          if href.startswith('http'):
              link = href
          else:
              # If it starts with '/', append the base URL
              link = urljoin(base_url, href)

          # Add the information to the result dictionary
          result_dict[text] = {'Link': link, 'Occurrences': 0}

  # Convert the dictionary to a JSON-formatted string
  json_string = json.dumps(result_dict, indent=2)

  #Print the JSON object
  #print('8787')
  return result_dict

def import_important_words():
  indexes_with_links = get_Important_words()
  # Get a list of 10 random indexes
  random_indexes = random.sample(indexes_with_links.keys(), 10)
  # Import the random indexes into Firebase
  for index in random_indexes:
      data = indexes_with_links[index]
      print(random_indexes)
      FBconn.put(indexes_table_name, index, data)
# import_important_words
#getAllPages()

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import display, HTML

class IndexTableUI:
    def __init__(self, index_data, firebase_ref):
        print('username',login_ui.username)
        self.index_data = index_data
        self.firebase_ref = firebase_ref

        self.tree = widgets.Output()

        self.populate_table()

        # Buttons for CRUD operations
        update_button = widgets.Button(description="Update", layout=widgets.Layout(width='auto'))
        add_button = widgets.Button(description="Add", layout=widgets.Layout(width='auto'))
        remove_button = widgets.Button(description="Remove", layout=widgets.Layout(width='auto'))

        update_button.style.button_color = '#20704a'  # Green color for Update button
        add_button.style.button_color = '#3498db'     # Blue color for Add button
        remove_button.style.button_color = '#e74c3c'  # Red color for Remove button

        update_button.on_click(self.update_row)
        add_button.on_click(self.add_row)
        remove_button.on_click(self.remove_row)

        buttons_hbox = widgets.HBox([update_button, add_button, remove_button])

        # Apply some margin to the buttons
        buttons_hbox.layout.margin = '10px 0'
        if login_ui.role == 'admin':
            display(widgets.VBox([self.tree, buttons_hbox]))
        else:
            display(widgets.VBox([self.tree]))


    def populate_table(self):
        with self.tree:
            clear_output(wait=True)

            # Creating an HTML table with CSS styles
            table_html = "<style>\
                           table {\
                             font-family: Arial, sans-serif;\
                             border-collapse: collapse;\
                             width: 70%;\
                           }\
                           th, td {\
                             border: 1px solid #dddddd;\
                             text-align: left;\
                             padding: 8px;\
                           }\
                           th {\
                             background-color: #20704a;\
                             color: white;\
                           }\
                           tr:nth-child(odd) {\
                             background-color: #1c5f3f;\
                           }\
                           </style>"

            table_html += "<table><tr><th>Index</th><th>Link</th></tr>"

            for word, data in self.index_data.items():
                link = f"<a href='{data['Link']}' target='_blank'>{data['Link']}</a>"
                occurrences = data['Occurrences']
                table_html += f"<tr><td>{word}</td><td>{link}</td></tr>"

            table_html += "</table>"
            display(HTML(table_html))

    def update_row(self, _):
        word_input = widgets.Text(description="Word:")
        link_input = widgets.Text(description="Link:")
        occurrences_input = widgets.IntText(description="Occurrences:")

        display(widgets.VBox([word_input, link_input, occurrences_input]))

        update_button = widgets.Button(description="Update Row")

        def update_row_callback(_):
            word = word_input.value
            link = link_input.value
            occurrences = occurrences_input.value

            if word in self.index_data:
                self.index_data[word]['Link'] = link
                self.index_data[word]['Occurrences'] = occurrences

                # Update occurrences in Firebase
                self.firebase_ref.put(indexes_table_name, word, {'Link': link, 'Occurrences': occurrences})
                #self.firebase_ref.child(word).update({'Link': link, 'Occurrences': occurrences})

            self.populate_table()

        update_button.on_click(update_row_callback)
        display(update_button)

    def add_row(self, _):
        word_input = widgets.Text(description="Word:")
        link_input = widgets.Text(description="Link:")
        # occurrences_input = widgets.IntText(description="Occurrences:")

        display(widgets.VBox([word_input, link_input]))

        add_button = widgets.Button(description="Add Row")

        def add_row_callback(_):
            word = word_input.value
            link = link_input.value
            # occurrences = occurrences_input.value

            if word not in self.index_data:
                self.index_data[word] = {'Link': link, 'Occurrences': 0}

                # Add new data to Firebase
                self.firebase_ref.put(indexes_table_name, word, {'Link': link, 'Occurrences': 0})


            self.populate_table()

        add_button.on_click(add_row_callback)
        display(add_button)

    def remove_row(self, _):
        word_input = widgets.Text(description="Word:")

        display(widgets.VBox([word_input]))

        remove_button = widgets.Button(description="Remove Row")

        def remove_row_callback(_):
            word = word_input.value

            if word in self.index_data:
                # Remove data from Firebase
                # Remove data from Firebase

                self.firebase_ref.delete(indexes_table_name, word)

                del self.index_data[word]

            self.populate_table()

        remove_button.on_click(remove_row_callback)
        display(remove_button)

# Example usage
def indexes_table_page():
     table_data = FBconn.get(indexes_table_name, None)
     app = IndexTableUI(table_data, FBconn)


In [5]:
!pip install mpld3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 4.2 MB/s eta 0:00:00


In [6]:
from IPython.core.display import display, HTML
import re
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output
import mpld3
import base64
from io import BytesIO

# Your existing functions...

def process_word_counts():
    # Reference to the original links data in Firebase
    links_data = FBconn.get('vmware_indexes', None)

    processed_word_counts = {}
    overall_word_counts = {}

    for topic, data in links_data.items():
        url = data['Link']
        soup = fetch_page(url)
        if soup:
            word_counts = index_words(soup)
            word_counts = remove_stop_words(word_counts)
            word_counts = apply_stemming(word_counts)

            # Update the overall word counts with the counts from this link
            for word, count in word_counts.items():
                overall_word_counts[word] = overall_word_counts.get(word, 0) + count

            # No longer saving each URL's word count to Firebase, but storing it in-memory for analysis
            processed_word_counts[url] = {"word_counts": word_counts}

    return processed_word_counts, overall_word_counts

def analyze_word_counts(processed_word_counts):
    overall_word_counts = {}
    all_links_words = set()
    most_used_word_per_link = {}
    first_link = True

    for url, data in processed_word_counts.items():
        word_counts = data.get("word_counts", {})

        # Aggregate overall word counts
        for word, count in word_counts.items():
            overall_word_counts[word] = overall_word_counts.get(word, 0) + count

        # Set operations for finding common words
        if first_link:
            all_links_words = set(word_counts.keys())
            first_link = False
        else:
            all_links_words &= set(word_counts.keys())

        # Finding most used word per link
        if word_counts:
            most_used_word = max(word_counts.items(), key=lambda item: item[1])
            most_used_word_per_link[url] = most_used_word

    # The rest of the analysis remains unchanged
    top_5_words = sorted(overall_word_counts.items(), key=lambda item: item[1], reverse=True)[:5]
    single_count_words = sum(1 for count in overall_word_counts.values() if count == 1)
    common_words = list(all_links_words)

    # Compile results into a dictionary
    analysis_results = {
        "top_5_words": top_5_words,
        "single_count_words": single_count_words,
        "common_words": common_words,
        "most_used_word_per_link": most_used_word_per_link
    }

    return analysis_results
processed_word_counts, _ = process_word_counts()
results = analyze_word_counts(processed_word_counts)

import matplotlib.pyplot as plt
import base64
from io import BytesIO


def plot_top_5_words_and_return_base64(top_5_words):
    if not top_5_words:  # If there's no data, skip plotting
        return None
    words, counts = zip(*top_5_words)
    plt.figure(figsize=(10, 6))
    plt.bar(words, counts, color='skyblue')
    plt.xlabel('Words')
    plt.ylabel('Counts')
    plt.title('Top 5 Most Counted Words')

    # Save the plot to a BytesIO object and encode it as a base64 string
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    plt.close()  # Close the plot to prevent it from displaying in the notebook
    buffer.seek(0)
    image_png = buffer.getvalue()
    buffer.close()

    base64_string = base64.b64encode(image_png).decode('utf-8')
    return base64_string


def plot_most_used_word_per_link_and_return_base64(most_used_word_per_link):
    if not most_used_word_per_link:  # If there's no data, skip plotting
        return None

    # Prepare the data
    urls = list(most_used_word_per_link.keys())
    words = [pair[0] for pair in most_used_word_per_link.values()]
    counts = [pair[1] for pair in most_used_word_per_link.values()]

    # Create the plot
    plt.figure(figsize=(10, 6))
    y_pos = range(len(urls))
    plt.barh(y_pos, counts, color='lightgreen', align='center')
    plt.yticks(y_pos, words)
    plt.xlabel('Counts')
    plt.title('Most Used Word in Each Link')

    # Add the URL as a text label next to each bar
    for i, url in enumerate(urls):
        plt.text(counts[i], i, f"{words[i]} ({url})", va='center')

    # Save the plot to a BytesIO object and encode it as a base64 string
    buffer = BytesIO()
    plt.savefig(buffer, format='png', bbox_inches="tight")
    plt.close()  # Close the plot to prevent it from displaying in the notebook
    buffer.seek(0)
    image_png = buffer.getvalue()
    buffer.close()

    base64_string = base64.b64encode(image_png).decode('utf-8')
    return base64_string

def statistics_page():
    # Execute the analysis and display the results
    processed_word_counts, _ = process_word_counts()
    results = analyze_word_counts(processed_word_counts)

    # Create an output widget for displaying the results
    stats_output = widgets.Output(layout=widgets.Layout(width='50%'))

    with stats_output:
        # Clear the output to ensure proper rendering
        clear_output(wait=True)

        # Display a title using HTML
        display(HTML("<h2>Word Count Analysis Results</h2>"))

        # Display the top 5 most counted words
        display(HTML("<h3>Top 5 Most Counted Words:</h3>"))
        top_5_words_base64 = plot_top_5_words_and_return_base64(results.get("top_5_words", []))
        if top_5_words_base64:
            display(HTML(f'<img src="data:image/png;base64,{top_5_words_base64}"/>'))
        else:
            display(HTML("<p>No data available for top 5 words.</p>"))

        # Display a section break using HTML
        display(HTML("<hr>"))

        # Display the most used word per link
        display(HTML("<h3>Most Used Word per Link:</h3>"))
        most_used_word_per_link_base64 = plot_most_used_word_per_link_and_return_base64(results.get("most_used_word_per_link", {}))
        if most_used_word_per_link_base64:
            display(HTML(f'<img src="data:image/png;base64,{most_used_word_per_link_base64}"/>'))
        else:
            display(HTML("<p>No data available for most used word per link.</p>"))

    # Display the output widget
    display(stats_output)


In [7]:
from firebase import firebase
FBconn = firebase.FirebaseApplication('https://cloudcomputing-2bbc8-default-rtdb.europe-west1.firebasedatabase.app/',None)

def insert_users():
   user_data = {
            'username': 'm',
            'password': '123',
            'role': 'user'
        }
   FBconn.post('/users',user_data)

# insert_users()

In [8]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import requests

def build_search_history(index, d):
    result = []
    urls = []
    for item in d:
        urls.append(item['Link'])
    if urls:
        result.append({'index': index, 'urls': urls})
    return result

# Function to update the Firebase dictionary
def update_search_history(username, search_history):
    database_url = 'https://cloudcomputing-2bbc8-default-rtdb.europe-west1.firebasedatabase.app/users.json'  # Adjust the URL to point to your database
    response = requests.get(database_url)
    data = response.json()

    if data:
        for key, value in data.items():
            if value.get('username') == username:
                if 'search_history' in value:
                    for item in search_history:
                        index = item['index']
                        urls = item['urls']
                        existing_index = next((x for x in value['search_history'] if x['index'] == index), None)
                        if existing_index:
                            existing_index['urls'] = list(set(existing_index['urls'] + urls))
                        else:
                            value['search_history'].append(item)
                else:
                    value['search_history'] = search_history

                # Update the entry in Firebase using HTTP PATCH request
                patch_url = f'{database_url[:-5]}/{key}.json'
                requests.patch(patch_url, json={"search_history": value['search_history']})
                return

def extract_indexes(dic):
    if dic != None:
        return [item['index'] for item in dic]
    else:
        return None

def get_search_history(username, user_dict):
    for key, value in user_dict.items():
        if value.get('username') == username and 'search_history' in value:
            return value['search_history']
    return None

def get_urls_for_index(index, dictionary):
    for item in dictionary:
        if item['index'] == index:
            return item['urls']
    return None


# Define search_output outside of the function
search_output = widgets.Output()

def create_search_page(user_name):
    # Function to handle search button click
    # Initialize Firebase connection
    FBconn = firebase.FirebaseApplication('https://cloudcomputing-2bbc8-default-rtdb.europe-west1.firebasedatabase.app/', None)

    def handle_search_button_click(b):
        query = search_input.value.strip()
        if query:
            search_results = vmwareSearch(query)
            search_values = build_search_history(query, search_results)
            update_search_history(user_name, search_values)
            search_history_values=["choose from list"]
            search_history_values+= extract_indexes(get_search_history(user_name,(FBconn.get('/users',None))))  # Sample search history values
            dropdown.options=search_history_values
            with search_output:
                  display_results(search_results)


        else:
            with search_output:
                clear_output()
                print("Please enter a search query.")

    # Function to generate dropdown widget for search history
    def generate_dropdown(search_history_values):
        dropdown = widgets.Dropdown(
            options=search_history_values,
            description='Search History:',
            style={'description_width': 'initial'}
        )
        return dropdown

    # Function to handle dropdown value change
    def handle_dropdown_change(change):
        picked_value = change['new']
        if picked_value != "choose from list":
            search_dic=get_search_history(user_name,FBconn.get('/users',None))
            with search_output:
              display_past_results(get_urls_for_index(picked_value,search_dic))

     # Function to display search results in a table
    def display_past_results(results):
        clear_output()  # Clear the output area before displaying new results
        cnt = 0
        table_html2 = f"""
        <style>
            table {{
                width: 80%; /* Adjust table width */
                margin: 20px auto; /* Center table */
                border-collapse: collapse;
                font-family: Arial, sans-serif;
            }}

            th, td {{
                padding: 10px;
                border: 1px solid #dddddd;
                text-align: left;
            }}

            th {{
                background-color: #4CAF50; /* Green header background */
                color: white;
            }}

            tr:nth-child(even) {{
                background-color: #f2f2f2; /* Light grey background for even rows */
            }}

            .search-container {{
                text-align: center;
                margin-bottom: 20px;
            }}

            .search-input {{
                width: 50%;
                padding: 10px;
                border-radius: 5px;
                border: 1px solid #cccccc;
                font-size: 14px;
                margin-bottom: 10px; /* Added margin between search input and button */
            }}

            .search-button {{
                width: 120px; /* Adjust button width */
                padding: 10px 20px;
                background-color: #4CAF50; /* Green button color */
                color: white;
                border: none;
                border-radius: 5px;
                cursor: pointer;
                font-size: 14px;
            }}

            .search-button:hover {{
                background-color: #45a049; /* Darker green on hover */
            }}

            .header {{
                text-align: center;
                margin-bottom: 20px;
                font-size: 24px;
                color: #4CAF50; /* Green color for the header */
            }}
        </style>
        <h2 class="header">Past Results</h2>
        <table>
            <tr>
                <th>Link Number</th>
                <th>Link</th>
            </tr>
        """
        for result in results:
            table_html2 += f"<tr><td>{cnt}</td><td>{result}</td><td></tr>"
            cnt += 1
        table_html2 += "</table>"
        display(HTML(table_html2))

    # Function to display search results in a table
    def display_results(results):
        clear_output()  # Clear the output area before displaying new results
        cnt = 0
        table_html = f"""
        <style>
            table {{
                width: 80%; /* Adjust table width */
                margin: 20px auto; /* Center table */
                border-collapse: collapse;
                font-family: Arial, sans-serif;
            }}

            th, td {{
                padding: 10px;
                border: 1px solid #dddddd;
                text-align: left;
            }}

            th {{
                background-color: #4CAF50; /* Green header background */
                color: white;
            }}

            tr:nth-child(even) {{
                background-color: #f2f2f2; /* Light grey background for even rows */
            }}

            .search-container {{
                text-align: center;
                margin-bottom: 20px;
            }}

            .search-input {{
                width: 50%;
                padding: 10px;
                border-radius: 5px;
                border: 1px solid #cccccc;
                font-size: 14px;
                margin-bottom: 10px; /* Added margin between search input and button */
            }}

            .search-button {{
                width: 120px; /* Adjust button width */
                padding: 10px 20px;
                background-color: #4CAF50; /* Green button color */
                color: white;
                border: none;
                border-radius: 5px;
                cursor: pointer;
                font-size: 14px;
            }}

            .search-button:hover {{
                background-color: #45a049; /* Darker green on hover */
            }}

            .header {{
                text-align: center;
                margin-bottom: 20px;
                font-size: 24px;
                color: #4CAF50; /* Green color for the header */
            }}
        </style>
        <h2 class="header">Search Results</h2>
        <table>
            <tr>
                <th>Link Number</th>
                <th>Link</th>
                <th>Search Result</th>
            </tr>
        """
        for result in results:
            if result["searchResult"]:
                table_html += f"<tr><td>{cnt}</td><td>{result['Link']}</td><td>{result['searchResult']}</td></tr>"
                cnt += 1
        table_html += "</table>"
        display(HTML(table_html))

    # Create search input widget
    search_input = widgets.Text(placeholder='Enter search query', layout=widgets.Layout(width='50%', description='Search Query'))

    # Create search button widget
    search_button = widgets.Button(description='Search', layout=widgets.Layout(background_color='#4CAF50', color='white', border='none', border_radius='5px', cursor='pointer', font_size='14px'))

    # Attach handle_search_button_click function to search button click event
    search_button.on_click(handle_search_button_click)

    # Display widgets
    searchList=extract_indexes(get_search_history(user_name,(FBconn.get('/users',None))))
    search_history_values=["choose from list"]
    if searchList != None:
        search_history_values+= searchList
    dropdown = generate_dropdown(search_history_values)
    dropdown.observe(handle_dropdown_change, names='value')
    display(widgets.VBox([widgets.HTML("<h1>Search Page</h1><h3>Results will be sorted from highest counter of query words to the lowest.</h3>"), search_input, search_button, search_output, dropdown]))


# Call the create_search_page function to create and display the search page
#create_search_page('m')

In [9]:


import nltk
from nltk.chat.util import Chat, reflections

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
# Define some patterns and responses
  ################################
##info for the indexes from the website for chatbot
from bs4 import BeautifulSoup
def extract_content_from_url(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
  }
    response = requests.get(url,headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        h1_tags = soup.find_all('h1')
        for h1_tag in h1_tags:
            # Find the next sibling div tag
            div_tag = h1_tag.find_next_sibling('div')
            if div_tag:
                # Find the first p tag under the div tag
                p_tag = div_tag.find('p')
                if p_tag:
                    return(p_tag.text.strip())
                else:
                    return("No p tag found under", h1_tag.text.strip())
            else:
                return("No div tag found after", h1_tag.text.strip())


    else:
        print("Failed to fetch webpage")
        return None



##the pattern to chatbot

from firebase import firebase
firebase = firebase.FirebaseApplication('https://cloudcomputing-2bbc8-default-rtdb.europe-west1.firebasedatabase.app', None)
result = firebase.get('/vmware_indexes', None)
a=[]
l=[]

for key,Link in result.items():
    a.append(key)
    l.append(Link['Link'])

print(a)
patterns = [
    (r'hi|hello', ['hi!', 'hello!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'(.*) your name?', ['You can call me Chatbot.', 'I go by the name Chatbot.']),
    (r'exit|bye', ['goodbye','bye','have a good day']),
    (r'what can you do\?', ['I can assist you with various tasks, such as answering questions, providing information, or just having a conversation.']),
    (r'help(.*)', ['Sure, what do you need help with?', 'How can I assist you?']),
    (r'thank you(.*)|(.*)thanks(.*)', ['You\'re welcome!', 'No problem!', 'Happy to help!']),
    (r'who created you\?', ['I was created by the developers muhammed fozi firas mustafa and tamer.']),
    (r'what is your purpose\?', ['My purpose is to assist and interact with users like you.']),


  ]
for key, link in zip(a, l):
    pattern = r'(?i)(.*)link(.*){}(.*)|(.*){}(.*)link(.*)'.format(re.escape(key), re.escape(key))
    patterns.append((pattern, [link]))
for key, link in zip(a, l):
    patterns.append((r'(.*){}(.*)'.format(key, key), [extract_content_from_url(link)]))

# import re
# # Create patterns for each index in array 'a'
# for index in a:
#     # Create a pattern that matches the index regardless of case
#     pattern = r'(.*){}(.*)'.format(re.escape(index), re.IGNORECASE)
#     patterns.append((pattern, [index]))

#take the words from the website to add to the chat box pattern





['Cloud Director', 'Cloud Provider Platform', 'Dell EMC VxRail', 'Horizon Cloud on Microsoft Azure', 'NSX Advanced Threat Prevention', 'Secure Remote Access', 'Telco Cloud Service Assurance', 'VMware Lab Platform', 'Workspace ONE Assist', 'Workspace ONE Freestyle Orchestrator']


In [11]:

  # Create a chatbot
chatbot = Chat(patterns, reflections)


import ipywidgets as widgets
from IPython.display import display




class ChatUI:
    def __init__(self):
        self.label = widgets.Label(value="Hello! I'm ChatGPT. How can I help you today?")
        self.labelHistory = widgets.Label(value="History: ")
        self.user_input = widgets.Text(placeholder='Type your message here...')
        self.conversation = widgets.Output()
        self.send_button = widgets.Button(description='Send')
        self.send_button.on_click(self.send_message)

        # Get chat history from Firebase
        res = FBconn.get('/chatbot_history/'+login_ui.user_id, None)
        chatbot_history = []
        if res:
            chatbot_history = list(res.values())

        # Create a label widget for displaying chat history
        self.history_label = widgets.HTML(value=self.format_chat_history(chatbot_history))

        self.chatbot = Chatbot()
        # Display a note with a few indexes from the 'a' array
        note_text = "Choose to ask questions about topics from index page or others such as : "
        indexes_to_display = a[:3]  # Display the first 3 indexes
        note_text += ", ".join(indexes_to_display)
        self.note_label = widgets.Label(value=note_text + ',..')

        display(widgets.VBox([self.note_label,self.labelHistory, self.history_label ,self.label, self.conversation, self.user_input, self.send_button ]))

    def format_chat_history(self, chat_history):
        # Format the chat history with HTML for better display
        formatted_history = "<div style='font-family: Arial; color: seagreen;'>"
        for entry in chat_history:
            formatted_history += f"<p>{entry}</p>"
        formatted_history += "</div>"
        return formatted_history





    def send_message(self, _):
        user_input = self.user_input.value
        chat = []
        with self.conversation:
            print('You:', user_input)
            FBconn.post('/chatbot_history/'+login_ui.user_id, 'You: '+user_input)

            response = self.chatbot.respond(user_input)
            print("ChatGPT:", response)
            FBconn.post('/chatbot_history/'+login_ui.user_id, "ChatGPT: " +response)
        self.user_input.value = ''



class Chatbot:


      def respond(self, user_input):
          response = chatbot.respond(user_input)
          return response
          if user_input.lower() == 'exit' or user_input.lower() == 'bye':
              return





In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from firebase import firebase

# Initialize Firebase connection
FBconn = firebase.FirebaseApplication('https://cloudcomputing-2bbc8-default-rtdb.europe-west1.firebasedatabase.app/', None)

class LoginUI:
    def __init__(self):
        print('Admin user: mustafa, password: 123 \n Regular user: m, password 123')
        self.username_input = widgets.Text(placeholder='Username', description='Username:')
        self.password_input = widgets.Password(placeholder='Password', description='Password:')
        self.login_button = widgets.Button(description='Login', button_style='success')

        self.login_button.on_click(self.login_clicked)

        self.login_box = widgets.VBox([self.username_input, self.password_input, self.login_button])

        self.logout_button = widgets.Button(description='Logout', button_style='danger')
        self.logout_button.on_click(self.logout_clicked)

        self.top_right_box = widgets.HBox([self.logout_button])

        self.ui = widgets.VBox([self.top_right_box, self.login_box])
        self.toggle_logout_button(False)  # Initially hide the logout button
        display(self.ui)

        # Initialize username as None
        self.username = None
        self.role = None
        self.user_id = None
    def login_clicked(self, b):
        username = self.username_input.value.strip()
        password = self.password_input.value.strip()

        # Retrieve user data from Firebase
        user_data = FBconn.get('/users', None)

        if user_data:
            for user_id, data in user_data.items():
                if data['username'] == username and data['password'] == password:
                    self.user_id= user_id
                    self.username = data['username']
                    self.role = data['role']
                    if data['role'] == 'admin':
                        clear_output(wait=True)
                        tabbed_ui = TabbedUI()
                        self.toggle_logout_button(True)
                    else:
                        clear_output(wait=True)
                        tabbed_ui = TabbedUI(exclude_index=True)
                    self.toggle_logout_button(True)  # Show the logout button
                    return
        print("Login failed. Please enter valid credentials.")

    def logout_clicked(self, b):
        clear_output(wait=True)
        self.toggle_logout_button(False)  # Hide the logout button
        self.__init__()  # Reset the login UI

    def toggle_logout_button(self, visible):
        self.top_right_box.layout.visibility = 'visible' if visible else 'hidden'

search_output = widgets.Output()
class TabbedUI:
    def __init__(self, exclude_index=False,role='user'):

        self.tabs = widgets.Tab()

        # Create different UI sections for each tab
        self.tab1 = widgets.Output()
        self.tab2 = widgets.Output()
        self.tab3 = widgets.Output()
        self.tab4 = widgets.Output()

        self.tabs.children = [self.tab1, self.tab2, self.tab3, self.tab4]
        self.tabs.set_title(0, 'Index Page')
        self.tabs.set_title(1, 'Search')
        self.tabs.set_title(2, 'Statistics')
        self.tabs.set_title(3, 'Chatbot')

        # if exclude_index:
        #     self.tabs.children = [self.tab2, self.tab3, self.tab4]
        #     self.tabs.set_title(0, 'Search')
        #     self.tabs.set_title(1, 'Statistics')
        #     self.tabs.set_title(2, 'Chatbot')

        self.title_username = 'welcome '+ login_ui.username+'!'
        self.title_widget = widgets.Label(value=self.title_username, layout=widgets.Layout(display='flex', justify_content='center', align_items='center'))
        dashboard_Box = widgets.VBox([self.title_widget,self.tabs])
        # Display the tabs
        display(dashboard_Box)

        # Set the initial UI content for each tab
        with self.tab1:
            self.display_tab1_content()

        with self.tab2:
            self.display_tab2_content()

        with self.tab3:
            self.display_tab3_content()

        with self.tab4:
            self.display_tab4_content()

        # Set the event handlers for tab changes
        self.tabs.observe(self.on_tab_change, names='selected_index')

    def display_tab1_content(self):
        clear_output(wait=True)
        indexes_table_page()

    def display_tab2_content(self):
        clear_output(wait=True)
        create_search_page(login_ui.username)

    def display_tab3_content(self):
        clear_output(wait=True)
        statistics_page()

    def display_tab4_content(self):
        clear_output(wait=True)
        chat_ui = ChatUI()
        display(chat_ui)

    def on_tab_change(self, change):
        selected_index = change.new

        if selected_index == 0:
            with self.tab1:
                self.display_tab1_content()
        elif selected_index == 1:
            with self.tab2:
                self.display_tab2_content()
        elif selected_index == 2:
            with self.tab3:
                self.display_tab3_content()
        elif selected_index == 3:
            with self.tab4:
                self.display_tab4_content()

if __name__ == "__main__":
    login_ui = LoginUI()


Admin user: mustafa, password: 123 
 Regular user: m, password 123
